In [1]:
%load_ext autoreload
%autoreload 2
import gc
from glob import glob
import os
import re
import sys
import yaml
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import timer, get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename, logger_func
from ieee_train import eval_train, eval_check_feature
from kaggle_utils import reduce_mem_usage, move_feature
import shutil

In [37]:
feim = pd.read_csv('../output/valid_single_feature_tochi.csv', header=None)
# feim = pd.read_csv('../output/bear_valid_single_feature.csv', header=None)
feim.columns = ['datetime', 'DT-M', 'feature', 'score']
# feim.columns = ['datetime', 'DT-M', 'feature', 'score', 'pb', 'pv', 'all']

feim = feim[feim['datetime']>='20190916_00000']
print(feim['datetime'].max())

feim.set_index('feature', inplace=True)
cnt = feim.groupby('feature')['score'].count()
idx = cnt[cnt>=1].index
feim = feim.loc[idx]
may = feim[feim['DT-M']=='2018-5']
# may.sort_values(by='pb', ascending=False, inplace=True)
# feim.loc[may.index]
# feim.sort_values(by='pb', ascending=False)

20190916_16174


In [40]:
tmp = feim[feim['DT-M']=='2018-5']
list_feature = tmp[tmp['score']>0.9370].index
# list_feature = tmp[tmp['pb']>0.960].index
# list_feature = feim[feim['pb']>0.960].index
# list_feature = tmp[(0.937 < tmp['score']) & (tmp['score'] < 0.938)].index
# from_dir = 'product_feature'
from_dir = 'check_trush'
# from_dir = 'org_trush'
from_dir = 'create'
# to_dir = 'org_use'
to_dir = 'valid_use'

# tmp.sort_values(by='pb', ascending=False, inplace=True)
# display(tmp)

for feature in list_feature:
    print(feature)
    move_feature([feature.replace('_train', '')], from_dir, to_dir)

609__DT-M_2017-12_130__cnt_uid3_C14_V187_cnt_ratio_global_train
609__DT-M_2017-12_130__new_uid4_C14_V187_cnt_train
609__DT-M_2017-12_130__new_uid4_C1_C12_cnt_ratio_global_train
609__DT-M_2018-11_130__cnt_uid4_C1_cnt_train
609__DT-M_2018-1_130__cnt_uid4_C6_C14_cnt_train
609__DT-M_2018-2_130__cnt_uid3_C6_V258_cnt_train
609__DT-M_2018-2_130__cnt_uid4_C6_V258_cnt_ratio_global_train
609__DT-M_2018-2_130__new_uid4_C12_C14_cnt_ratio_global_train
609__DT-M_2018-3_130__cnt_uid3_C6_V187_cnt_ratio_global_train
609__DT-M_2018-3_130__cnt_uid4_C12_C14_cnt_train
609__DT-M_2018-3_130__cnt_uid4_C1_C12_cnt_ratio_global_train
609__DT-M_2018-3_130__new_uid4_C6_C12_cnt_train
609__DT-M_2018-4_130__new_uid4_C6_C14_cnt_train
609__DT-M_2018-5_130__cnt_uid3_C6_C12_cnt_ratio_global_train
609__DT-M_2018-5_130__new_uid4_C6_C12_cnt_train
609__DT-M_2018-7_130__cnt_uid4_C12_C14_cnt_train
609__DT-M_2018-8_130__cnt_uid3_C6_V187_cnt_train
609__DT-M_2018-8_130__cnt_uid4_C6_V258_cnt_train
609__DT-M_2018-9_130__cnt_uid2_cn

array([5])

In [20]:
tmp[tmp['score']>0.937]

,datetime,DT-M,score
feature,,,
509__C11-V44__ratio__ProductCD-C_train,20190914_06201,2018-5,0.937060
512__V127-V318__ratio__ProductCD-W_train,20190914_13211,2018-5,0.938651
512__V127-V78__diff__ProductCD-C_train,20190914_16115,2018-5,0.937358
512__V130-V281__diff__ProductCD-H_train,20190914_06140,2018-5,0.937180
512__V53-V78__diff__ProductCD-W_train,20190914_13155,2018-5,0.937122
603__addr1-addr2_C8__ProductCD-H_std_train,20190914_16063,2018-5,0.937199
603__card1-card4_C6__ProductCD-C_mean_train,20190914_07480,2018-5,0.937604
603__card3-card6_C9__ProductCD-C_std_train,20190914_06595,2018-5,0.937283
603__card6-addr2_C4__ProductCD-C_std_train,20190914_12505,2018-5,0.937022
